# Knowledge Distillation in MHIST Dataset

In [1]:
import os
import sys
import numpy as np
import random
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

plt.style.use('_mpl-gallery')

from Utilities import *

# Load dataset

In [2]:
# Prepare data
path = "mhist_dataset/images"
CSVfile = "mhist_dataset/annotations.csv"

data, file_names = load_mhist_images(path)
X_train, y_train, X_test, y_test = loadMHIST(CSVfile,data)

# Data Augmentation
AugmentedData = []
AugmentedLabel = []
for i in range(len(X_train)):
    if((y_train[i]==(1,0)).all()):
        AugmentedData.append(cv2.rotate(X_train[i], cv2.ROTATE_90_CLOCKWISE))
        AugmentedLabel.append([1,0])
X_train = np.concatenate((X_train, AugmentedData))
y_train = np.concatenate((y_train, AugmentedLabel))
c = list(zip(X_train, y_train))

random.shuffle(c)

X_train, y_train = zip(*c)
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

# Data Normalization
X_train = X_train-np.mean(X_train)/np.std(X_train)
X_test = X_test-np.mean(X_train)/np.std(X_train)

# Data Batching
Train_Data, Train_Label, Test_Data, Test_Label = dataBatching(X_train, y_train, X_test, y_test)


# Train Teacher Model (Resnet)

In [ ]:
resNetBase= tf.keras.applications.resnet_v2.ResNet50V2(
        include_top = False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling=None,
    )
for layer in resNetBase.layers[:]:
  layer.trainable = False
x = tf.keras.layers.Flatten()(resNetBase.output)
x = tf.keras.layers.Dense(2)(x)
restNet = tf.keras.Model(inputs=resNetBase.input, outputs=x)

## intial epochs at learning rate 0.0001
TrainResAcc1 = train_and_evaluate(restNet,Train_Data,Test_Data,Train_Label,Test_Label,10, 0.0001)

## Fine tune epochs at learning rate 0.00001
resNetBase.trainable = True
TrainResAcc2 = train_and_evaluate(restNet,Train_Data,Test_Data,Train_Label,Test_Label,25, 0.00001)


In [ ]:
TrainResAcc = TrainResAcc1 + TrainResAcc2

# Train Student Mobile Net using KD


In [ ]:
studenModel2 = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top = False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling=None,
    )
for layer in studenModel2.layers[:]:
  layer.trainable = False
x = tf.keras.layers.Flatten()(studenModel2.output)
x = tf.keras.layers.Dense(2)(x)
mobileNetKD = tf.keras.Model(inputs=studenModel2.input, outputs=x)

## intial epochs
TrainMobileKDAcc1 = train_and_evaluate_mobileNet_using_KD(mobileNetKD,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,4, 10, 0.001)

## Fine tune epochs
studenModel2.trainable = True
TrainMobileKDAcc2 = train_and_evaluate_mobileNet_using_KD(mobileNetKD,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,4, 25, 0.0001)

In [ ]:
TrainMobileKDAcc = TrainMobileKDAcc1 + TrainMobileKDAcc2

# Train another student model from scratch

In [ ]:
studenModel2 = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top = False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling=None,
    )
for layer in studenModel2.layers[:]:
  layer.trainable = False
x = tf.keras.layers.Flatten()(studenModel2.output)
x = tf.keras.layers.Dense(2)(x)
mobileNet = tf.keras.Model(inputs=studenModel2.input, outputs=x)

## intial epochs
TrainMobileAcc1 = train_and_evaluate(mobileNet,Train_Data,Test_Data,Train_Label,Test_Label,10, 0.001)
## Fine tune epochs
studenModel2.trainable = True
TrainMobileAcc2 = train_and_evaluate(mobileNet,Train_Data,Test_Data,Train_Label,Test_Label,25, 0.0001)


In [ ]:
TrainMobileAcc = TrainMobileAcc1 + TrainMobileAcc2

# Models Testing

In [10]:
# model testing. returning test accuracy, specificity and sensitivity
teacherAcc = testTransferedModel(restNet,Test_Data,Test_Label)
studentAcc = testTransferedModel(mobileNet,Test_Data,Test_Label)
studentAccKD = testTransferedModel(mobileNetKD,Test_Data,Test_Label)

model Testing Accuracy: 50.79%
model Testing Specificity: 35.28%
model Testing Sensitivity: 62.40%
model Testing Accuracy: 52.88%
model Testing Specificity: 26.11%
model Testing Sensitivity: 71.15%
model Testing Accuracy: 58.83%
model Testing Specificity: 11.67%
model Testing Sensitivity: 89.30%


# Temperature Vs Accuracy

In [ ]:
resNetBase= tf.keras.applications.resnet_v2.ResNet50V2(
        include_top = False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling=None,
    )
for layer in resNetBase.layers[:]:
  layer.trainable = False
x = tf.keras.layers.Flatten()(resNetBase.output)
x = tf.keras.layers.Dense(2)(x)
restNet = tf.keras.Model(inputs=resNetBase.input, outputs=x)

testF1Score = []
tempratureValues = [1, 2, 4, 16, 32, 64]
for temp in tempratureValues:
    studenModel2 = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top = False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling=None,
    )
    for layer in studenModel2.layers[:]:
        layer.trainable = False
    x = tf.keras.layers.Flatten()(studenModel2.output)
    x = tf.keras.layers.Dense(2)(x)
    st = tf.keras.Model(inputs=studenModel2.input, outputs=x)

    # initial epochs
    train_and_evaluate_mobileNet_using_KD(st,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,temp, 10, 0.001)

    # fine tuning
    studenModel2.trainable = True
    train_and_evaluate_mobileNet_using_KD(st,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,temp, 25, 0.0001)
    testF1Score += [testTransferedModel(st,Test_Data,Test_Label)]

In [ ]:
print(testF1Score)

In [ ]:
f, ax = plt.subplots(figsize=(16, 4))
ax.plot(TrainResAcc, label = "Teacher model")
ax.plot(TrainMobileKDAcc, label = 'Student model trained using KD')
ax.plot(TrainMobileAcc, label = 'Student model trained from scratch')
ax.set_xlabel('Epoch', fontsize = 16)
ax.set_ylabel('Accuracy (%)', fontsize = 16)
ax.set_title(' Training Accuracy', fontsize = 20)
ax.legend()
plt.show()
f.savefig('Figures/fig3.png', bbox_inches='tight')



In [ ]:
f, ax = plt.subplots(figsize=(16, 4))
tempratureValues = [1, 2, 4, 16, 32, 64]
# F1 Score calculated for test data
ax.plot(tempratureValues, testF1Score)
ax.set_xlabel('Temperature', fontsize = 16)
ax.set_ylabel('F1 Score (%)', fontsize = 16)
ax.set_title('F1 Score of Student Model trained using KD vs Temperature', fontsize = 20)
ax.legend()
plt.show()
f.savefig('Figures/fig11.png', bbox_inches='tight')

In [ ]:
# your code start from here for step 8
from keras_flops import get_flops
TeacherParameters = restNet.summary()
StudentParameters = mobileNetKD.summary()

flops = get_flops(restNet, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

flops = get_flops(mobileNetKD, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")